In [11]:
# !pip install transformers
import numpy as np
import pandas as pd

from tokenizers import AddedToken
from transformers import T5Tokenizer

In [12]:
data_path="data/datafinal.json"
ftraind_path="data/f_traind.csv"
fdata_path="data/f_data.csv"


submission_path="data/submission.csv"

# vars
col_text='Text'
col_cc='ContainsCode'
col_cl='CodeList'


In [13]:
full_data=pd.read_json(data_path)
ftrain_data=pd.read_csv(ftraind_path)
f_data=pd.read_csv(ftraind_path)

train_data=full_data[full_data[col_cc]!=""]
test_data=full_data[full_data[col_cc]==""]
# pprint(train_data.iloc[1,1])
train_data
full_data.iloc[10]
# full_data.to_csv("/home/u131168/mh_shell/data/datafinal.csv")

ID                                                             11
Text            The field of software development is incredibl...
ContainsCode                                                 True
CodeList        #include <iostream> using namespace std; int m...
Name: 10, dtype: object

In [14]:
mh_dir='mh_shell'

In [15]:
# !pip install bitsandbytes
# !pip install accelerate
# !pip install scipy
# !pip install transformers
# !pip install peft

In [16]:
def getTokenizer():
    
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
    # stokens_v1=["`","~","!","@","#","$","%","^","&","*","(",")","-","_","+","=","{","}","[","]","|","\\",":",";","\"","'","<",">","?","/","\n","\t"," "]
    # stokens_v2=["{","}","<","<<"]
    stokens_v3=["{","}","<","`","\\"]

    stokens=stokens_v3
    for st in stokens:
        tokenizer.add_tokens(AddedToken(st, normalized=False),special_tokens=False)
    return tokenizer
    
    

In [17]:


from peft import AutoPeftModelForSeq2SeqLM
from transformers import AutoTokenizer
import torch
# model_path="google/flan-t5-xl"
# model_path=f"/home/u131168/{mh_dir}/ft_models/flan-t5-xl_peft_finetuned_model/checkpoint-11700"
# model_path=f"/home/u131168/{mh_dir}/ft_models/flan-t5-xl_mt5/checkpoint-34600"

model_path=f"/home/u131168/{mh_dir}/ft_models/flan-t5-xl_mt5_v3/checkpoint-25000"




custom_model = AutoPeftModelForSeq2SeqLM.from_pretrained(model_path, )

# tokenizer = T5Tokenizer.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

tokenizer=getTokenizer()
custom_model.resize_token_embeddings(len(tokenizer))


In [ ]:
from pprint import pprint

# p_count=50
# psi,pei=58,68
psi,pei=11,12

# psi,pei=250,258
# prompts = ftrain_data.loc[psi:pei,['input','instruction']].values.tolist()
prompts = f_data.loc[psi:pei,['input','instruction']].values.tolist()

# instruction="extract code snippets from the paragraph, if it does not contain code snippets then say none"

t_prompts=[]
for p in prompts:
    context=str(p[0])
    question=p[1]
    t_prompts.append([context, question])
    # t_prompts+=[f"input: {context}\n\ninstruction: {question}"]

prompts=t_prompts
# prompts = ftrain_data.loc[:p_count,['input','instruction']].values.tolist()
# pprint(prompts)

: 

: 

In [ ]:



# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


# stokens=[]
# stokens=["{", "}" ,"\n","<", ">"," ",'\\','/','^']
# for st in stokens:
    # tokenizer.add_tokens(AddedToken(st, normalized=False))


# tokenizer.add_tokens(stokens,special_tokens=False)


input_ids = tokenizer(prompts, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# sample up to 30 tokens
torch.manual_seed(0)  # doctest: +IGNORE_RESULT
outputs = custom_model.generate(input_ids=input_ids, do_sample=True, max_length=512)
# pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True,spaces_between_special_tokens = False)
# pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True)


: 

In [ ]:
pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True,spaces_between_special_tokens = False)

# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
# stokens=["`","~","!","@","#","$","%","^","&","*","(",")","-","_","+","=","{","}","[","]","|","\\",":",";","\"","'","<",">","?","/","\n"," "]
# for st in stokens:
#     tokenizer.add_tokens(AddedToken(st, normalized=False),special_tokens=False)

# pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True,spaces_between_special_tokens = False)


: 

In [ ]:
# # # !pip install transformers
# # from transformers import AutoTokenizer
# # from transformers import T5Tokenizer

# # from tokenizers import AddedToken
# text="int arr[5] = \n\{`~1,};:[ ]\++++ - *&^%$#@!_+)'5>2,"
# # # text="int arr[5] = {1, 2, 3, 4, 5};', 'for(int i=0; i<5; i++) {cout << arr[i] << endl;}"
# # # text="function add(a, b) { \n\n<< return a + b; }"

# # # tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
# # # tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# # stokens=[]
# # stokens=["{", "}" ,"\n","<",'\\','/','^'," ","`","~"]
# stokens=["`","~","!","@","#","$","%","^","&","*","(",")","-","_","+","=","{","}","[","]","|","\\",":",";","\"","'","<",">","?","/","\n"," "]
# for st in stokens:
#     tokenizer.add_tokens(AddedToken(st, normalized=False),special_tokens=False)

# # # tokenizer.add_tokens(["{","}","<","\n"])
# input_ids = tokenizer(text, return_tensors="pt" ,padding=True,truncation=True, max_length=512).input_ids
# # # pred_custom=tokenizer.batch_decode(outputs, skip_special_tokens=True)
# print([text])
# print(tokenizer.batch_decode(input_ids,skip_special_tokens=True,spaces_between_special_tokens = False))
# print(tokenizer.batch_decode(input_ids,skip_special_tokens=True,))



: 

In [ ]:
pred_custom_df=pd.DataFrame(pd.Series(pred_custom,name="pred_custom"))
# pred_ft5_df=pd.DataFrame(pd.Series(pred_ft5,name="pred_ft5"))
# temp_df=pd.concat([ftrain_data.iloc[psi:pei+1].reset_index(),pred_custom_df],axis=1)
temp_df=pd.concat([f_data.iloc[psi:pei+1].reset_index(),pred_custom_df],axis=1)

temp_df
# pprint(temp_df.iloc[1,0])
# print(temp_df.iloc[5,3])
# print(temp_df.iloc[5,4])

: 

In [ ]:
temp_df.iloc[8,1]

: 

In [ ]:
# # !pip install tiktoken
# # !pip install transformers
# # model='Salesforce/codegen-350M-mono'
# # model='Salesforce/codegen25-7b-instruct'
# model="Phind/Phind-CodeLlama-34B-v1"
# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model)


# # from transformers import AutoTokenizer, LlamaForCausalLM
# # model_path = "Phind/Phind-CodeLlama-34B-v1"
# # model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# # tokenizer = AutoTokenizer.from_pretrained(model_path)




: 

In [ ]:
# from pprint import pprint
# # creating prompts
# prompts=[]
# for i in train_data[124:125].index:
#     prompt='extract code from the following paragraph:\n\n\n'+train_data.iloc[i,1].replace('\n','')
    
#     # predict
#     # text = "def hello_world():"
#     # text = prompt
#     # input_ids = tokenizer(text, return_tensors="pt").input_ids
#     # generated_ids = model.generate(input_ids, max_length=128)
#     # res=tokenizer.decode(generated_ids[0], skip_special_tokens=True)

#     tokenizer.pad_token = tokenizer.eos_token
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

#     # Generate
#     generate_ids = model.generate(inputs.input_ids, max_new_tokens=256, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
#     completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     completion = completion.replace(prompt, "").split("\n\n\n")[0]
#     res=completion

#     print('-----------------------text----------------')
#     pprint(prompt)
#     print('-------------------------res------------------')
#     pprint(res)

    


: 